# API Course 4.1 - Overview
**Who is this for?** Beginners with basic C# skills looking to understand common REST API modules and patterns in ASP.NET Core.

**What you’ll learn:**
- Overview of core modules in version 4.1 (GettingStarted, BeyondTheBasics)
- Dependency Injection and service registration
- Swagger/OpenAPI integration
- Model validation filters and middleware error handling


# API Course 4.1 - Overview
This notebook explores the modules in the REST API Course version 4.1 and explains key concepts and code snippets in depth.

## Modules in Version 4.1
The `rest-api-course-final-master` folder contains two modules:
1. `1.GettingStarted`
2. `2.BeyondTheBasics`

## 1) Getting Started Module
This module introduces the solution structure and basic setup.

In [ ]:
!tree -L 2 /home/mzmzeeee/coding/C#AndDotnet/API_course/4.1/rest-api-course-final-master/1.GettingStarted

The projects mirror version 3.1 but may include new package versions or scaffold improvements. Review the `.csproj` files for changes in target frameworks.

### Example: Updated `Movies.Application` Interface
Notice how new methods for paging are added:

In [ ]:
namespace Movies.Application.Interfaces {
    public interface IMovieService {
        Task<IEnumerable<GetAllMoviesResponse>> GetAllAsync(int page = 1, int pageSize = 10);
        Task<GetMovieResponse> GetByIdAsync(int id);
        Task<int> CreateAsync(CreateMovieRequest request);
    }
}

- `page` and `pageSize` parameters enable server-side pagination.
- `GetByIdAsync` returns a single DTO.
- `CreateAsync` returns the created resource's ID.

## 2) Beyond The Basics Module
In this module, the course covers filters, swagger integration, and custom middleware.

In [ ]:
!tree -L 1 /home/mzmzeeee/coding/C#AndDotnet/API_course/4.1/rest-api-course-final-master/2.BeyondTheBasics

### Key Topics Covered
- OpenAPI/Swagger setup (`Swashbuckle.AspNetCore`).
- Model validation filters (`IActionFilter`).
- Global exception handling middleware.

#### Example: Registering Swagger in `Startup.cs`
```csharp
services.AddSwaggerGen(c => {
    c.SwaggerDoc("v1", new OpenApiInfo { Title = "Movies API", Version = "v1" });
});
app.UseSwagger(); app.UseSwaggerUI(c => { c.SwaggerEndpoint("/swagger/v1/swagger.json", "Movies API v1"); });
```

### Model Validation Filters
The custom `ValidationFilter` implements `IActionFilter` to intercept model binding errors and return a consistent `BadRequest` result when the model state is invalid.

```csharp
public class ValidationFilter : IActionFilter
{
    public void OnActionExecuting(ActionExecutingContext context)
    {
        if (!context.ModelState.IsValid)
        {
            context.Result = new BadRequestObjectResult(context.ModelState);
        }
    }
    public void OnActionExecuted(ActionExecutedContext context) { }
}
```

**Registration in Startup.cs**
```csharp
services.AddControllers(options =>
{
    options.Filters.Add<ValidationFilter>();
});
```

### Custom Exception Handling Middleware
A middleware component catches unhandled exceptions and returns a standardized error response.

```csharp
public class ExceptionMiddleware
{
    private readonly RequestDelegate _next;
    public ExceptionMiddleware(RequestDelegate next) => _next = next;
    public async Task InvokeAsync(HttpContext context)
    {
        try
        {
            await _next(context);
        }
        catch (Exception ex)
        {
            context.Response.StatusCode = 500;
            await context.Response.WriteAsJsonAsync(new { error = ex.Message });
        }
    }
}
```

**Use in `Startup.cs`**
```csharp
app.UseMiddleware<ExceptionMiddleware>();
```

## Dependency Injection Setup
In `Program.cs` (or `Startup.cs`), register controllers and application services:
```csharp
var builder = WebApplication.CreateBuilder(args);
builder.Services.AddControllers();
builder.Services.AddScoped<IMovieService, MovieService>();
var app = builder.Build();
app.MapControllers();
app.Run();
```